In [20]:
# from datetime import datetime
# import time
import streamlit as st
from streamlit_chat import message
# import pandas as pd
# import numpy as np
# import json
import os
from dotenv import load_dotenv
import openai
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader, PyPDFDirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate, MessagesPlaceholder
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.messages import HumanMessage


In [2]:
# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

/Users/lydiayu/opt/anaconda3/envs/artisan-ai/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

/Users/lydiayu/opt/anaconda3/envs/artisan-ai/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
pdf_loader = PyPDFDirectoryLoader('./books/')
txt_loader = DirectoryLoader('./books/', glob="**/*.txt", loader_cls=TextLoader)

In [6]:
txts = txt_loader.load()

In [7]:
txts

[Document(page_content='Shipping\n- Orders more than $75 will be mailed for free!\n- Orders over $20, that can be shipped in a single package, will be mailed via United Parcel Service (UPS) 3 Day Select® or United States Postal Service (USPS) Priority Mail.\n- Orders under $20 will be mailed via USPS Media Mail® (delivery, 2 to 8 business days).\n- We do not ship books outside the United States.\n\nOrder Status Descriptions\nAwaiting Fulfillment:\nThis is the initial state of your order. Your payment has cleared and you have been provided with an order number. Your order should not remain in this state for more than one business day.\nAwaiting Shipment:\nThe books in your order have been secured and are in the process of being mailed to you. At this stage we cannot refund an order unless the book(s) are returned to us.\n\nOrders placed before 1 p.m. will enter Awaiting Shipment status the same business day. Orders placed 1 p.m. will enter Awaiting Shipment status the following business

In [8]:
docs = pdf_loader.load()

In [9]:
docs.extend(txts)

In [10]:
docs

[Document(page_content='This is a sample document to\nshowcase page-based formatting. It\ncontains a chapter from a Wikibook\ncalled Sensory Systems . None of the\ncontent has been changed in this\narticle, but some content has been\nremo ved.Anat omy of the Somat osensor y System\nFROM WIKIBOOKS1\nOur somatosensor ysystem consists ofsensors intheskin\nand sensors inour muscles, tendons, and joints. The re-\nceptors intheskin, thesocalled cutaneous receptors, tell\nusabout temperature (thermorec eptors ),pressure and sur-\nfacetexture (mechano receptors ),and pain (nocic eptors ).\nThe receptors inmuscles and joints provide information\nabout muscle length, muscle tension, and joint angles.\nCutaneous r ecept ors\nSensor yinformation from Meissner corpuscles and rapidly\nadap ting afferents leads toadjustment ofgrip forcewhen\nobjec tsare lifted. These afferents respond with abrief\nburst ofaction potentials when objec tsmoveasmall dis-\ntanc eduring the early stages oflifting. Inrespo

In [11]:
# split docs into chunks of 1000 tokens
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs)

In [12]:
# ingest docs into faiss to efficiently query embeddings
db = FAISS.from_documents(docs, embeddings)

In [13]:
db

In [14]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


In [17]:
from langchain.chains import create_history_aware_retriever

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, db.as_retriever(), contextualize_q_prompt
)

In [18]:

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [22]:
chat_history = []

In [17]:
user_query = "find me a book about machine learning."

In [23]:
question = "find me a book about machine learning."
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])